In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
from splinter import Browser
import requests
# import pymongo

import tweepy
import json
from config import consumer_key, consumer_secret, access_token, access_token_secret

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

/usr/local/bin/chromedriver


In [3]:
# NASA Mars News

# Scrape the NASA Mars News Site and collect the latest News Title and 
# Paragraph Text. Assign the text to variables that you can reference 
# later.

In [4]:
# URL of page to be scraped
url1 = 'https://mars.nasa.gov/news/'
browser.visit(url1)

html = browser.html
soup = bs(html, 'html.parser')

In [5]:
# save the most recent article, title and date
article = soup.find("div", class_="list_text")
news_p = article.find("div", class_="article_teaser_body").text
news_title = article.find("div", class_="content_title").text
news_date = article.find("div", class_="list_date").text
print(news_date)
print(news_title)
print(news_p)

August 24, 2018
Digital Creators: Apply for NASA Mars Landing Event
Up to 30 multimedia participants will be selected for a special two-day event at JPL culminating in the InSight spacecraft's landing on Mars. Apply by Sept. 3.


In [6]:
# JPL Mars Space Images - Featured Image

# Visit the url for JPL Featured Space Image here.

# Use splinter to navigate the site and find the image url for the current
# Featured Mars Image and assign the url string to a variable called 
# featured_image_url.

# Make sure to find the image url to the full size .jpg image.

# Make sure to save a complete url string for this image.

In [7]:
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)

In [8]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# Find link to image
image = soup.find("a", class_="button fancybox")["data-fancybox-href"]

# Set full link to variable
featured_image_url = "https://jpl.nasa.gov"+image


print(featured_image_url)

https://jpl.nasa.gov/spaceimages/images/mediumsize/PIA17046_ip.jpg


In [9]:
# Mars Weather

# Visit the Mars Weather twitter account here and scrape the latest Mars 
# weather tweet from the page. Save the tweet text for the weather report 
# as a variable called mars_weather.


In [10]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [11]:
# Target User
target_user = "marswxreport"

# Pull latest tweet
latest_tweet = api.user_timeline(target_user, count = 1)

# Set to variable 
mars_weather = latest_tweet[0]['text']

print("Current Mars weather: " + mars_weather)

Current Mars weather: Sol 2147 (2018-08-21), high -15C/5F, low -68C/-90F, pressure at 8.70 hPa, daylight 05:30-17:44


In [12]:
# Mars Facts

# Visit the Mars Facts webpage here and use Pandas to scrape the table 
# containing facts about the planet including Diameter, Mass, etc.

# Use Pandas to convert the data to a HTML table string.

In [19]:
url3 = 'https://space-facts.com/mars/'
tables = pd.read_html(url3)
type(tables)
df = tables[0]
df.columns = ['Mars', 'Data']
df.set_index('Mars')
df

,Mars,Data
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [14]:
# Convert to html table
mars_table = df.to_html()
mars_table.replace('\n', '')
df.to_html('table.html')
!open table.html

In [15]:
# Mars Hemispheres

# Visit the USGS Astrogeology site here to obtain high resolution 
# images for each of Mar's hemispheres.

# You will need to click each of the links to the hemispheres in 
# order to find the image url to the full resolution image.

# Save both the image url string for the full resolution hemisphere 
# image, and the Hemisphere title containing the hemisphere name. 
# Use a Python dictionary to store the data using the keys img_url 
# and title.

# Append the dictionary with the image url string and the hemisphere 
# title to a list. This list will contain one dictionary for each 
# hemisphere.

In [16]:
url4 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url4)
html = browser.html
soup = bs(html, 'html.parser')

In [17]:
mars_hemispheres = []

for i in range (4):
    images = browser.find_by_tag('h3')
    images[i].click()
    html = browser.html
    soup = bs(html, 'html.parser')
    partial = soup.find("img", class_ ="wide-image")["src"]
    img_title = soup.find("h2",class_="title").text
    img_url = 'https://astrogeology.usgs.gov'+ partial
    dictionary={"title":img_title,"img_url":img_url}
    mars_hemispheres.append(dictionary)
    browser.back()

In [18]:
print(mars_hemispheres)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
